In [14]:
 
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from scipy.stats import mode
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


def prdict_heart_disease(list_data):
    df = pd.read_csv('/home/amityadav/Desktop/HealthDesease/media/Testing.csv')

#     X = df[['age','sex','cp',  'trestbps',  'chol',  'fbs',  'restecg',  'thalach',  'exang',  'oldpeak',  'slope',  'ca',  'thal']]
    X = df.iloc[:,:-1]
    y = df[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.808, random_state=0)
    nn_model = GradientBoostingClassifier(n_estimators=100,learning_rate=1.0,max_depth=1, random_state=0)
    final_svm_model = SVC()
    final_nb_model = GaussianNB()
    final_knn_model = KNeighborsClassifier(n_neighbors=5)
    final_svm_model.fit(X_train, y_train)
    final_nb_model.fit(X_train, y_train)
    final_knn_model.fit(X_train, y_train)
    nn_model.fit(X_train, y_train)
    pred = nn_model.predict([list_data])
    pred1 = final_svm_model.predict([list_data])
    pred2 = final_nb_model.predict([list_data])
    pred5 = final_knn_model.predict([list_data])
    print("Neural Network Accuracy: {:.2f}%".format(nn_model.score(X_test, y_test) * 100))
    print("SVM Accuracy: {:.2f}%".format(final_svm_model.score(X_test, y_test) * 100))
    print("Naive Bayes Accuracy: {:.2f}%".format(final_nb_model.score(X_test, y_test) * 100))
    print("Knn Accuracy: {:.2f}%".format(final_knn_model.score(X_test, y_test) * 100))
    print("Prdicted Value is : ", format(pred))
    print("Prdicted Value is 1: ", format(pred1))
    print("Prdicted Value is 2: ", format(pred2))
    print("Prdicted Value is 5: ", format(pred5))
    dataframe = str(df.head())
    d = {'GradientBoosting classifier':" {:.2f}%".format(nn_model.score(X_test, y_test) * 100),'SupportVector Classifier':" {:.2f}%".format(final_svm_model.score(X_test, y_test) * 100),'NaiveBayes Classifier':"{:.2f}%".format(final_nb_model.score(X_test, y_test) * 100),'KNearestNeighbor Classifier':"{:.2f}%".format(final_knn_model.score(X_test, y_test) * 100)}

    return (d,(pred))

 
 

         
def add_genralhealth(request):
    predictiondata = None
    symptomList = []           # selected symptoms
    
    if request.method=="POST":
        for i,j in request.POST.items():
            if "csrfmiddlewaretoken" != i:
                symptomList.append(i)
        # training.csv
        DATA_PATH = Admin_Helath_CSV.objects.get(id=2)
        data = pd.read_csv(DATA_PATH.csv_file).dropna(axis = 1)

        # Checking whether the dataset is balanced or not
        disease_counts = data["prognosis"].value_counts()
        temp_df = pd.DataFrame({
            "Disease": disease_counts.index,
            "Counts": disease_counts.values
        })

        encoder = LabelEncoder()
        data["prognosis"] = encoder.fit_transform(data["prognosis"])   # encoding


        X = data.iloc[:,:-1]    # rows except last column
        y = data.iloc[:, -1]    #last column
        X_train, X_test, y_train, y_test =train_test_split(
        X, y, test_size = 0.2, random_state = 24)
        symptoms = X.columns.values  
        symptom_index = {}
        for index, value in enumerate(symptoms):
            symptom = " ".join([i.capitalize() for i in value.split("_")])
            symptom_index[symptom] = index
        
        data_dict = {
            "symptom_index":symptom_index,
            "predictions_classes":encoder.classes_    # array of disease
        }

        final_svm_model = SVC()
        final_nb_model = GaussianNB()
        final_rf_model = RandomForestClassifier(random_state=18)
        final_ds_model = DecisionTreeClassifier(random_state=20)
        final_knn_model = KNeighborsClassifier(n_neighbors=5)
        final_svm_model.fit(X, y)
        final_nb_model.fit(X, y)
        final_rf_model.fit(X, y)
        final_ds_model.fit(X,y)
        final_knn_model.fit(X,y)
        

        #Testing.csv
        DATA_PATH2 = Admin_Helath_CSV.objects.get(id=3)
        test_data = pd.read_csv(DATA_PATH2.csv_file).dropna(axis=1)

        test_X = test_data.iloc[:, :-1]
        test_Y = encoder.transform(test_data.iloc[:, -1])

        svm_preds = final_svm_model.predict(test_X)
        nb_preds = final_nb_model.predict(test_X)
        rf_preds = final_rf_model.predict(test_X)
        ds_preds = final_ds_model.predict(test_X)
        knn_preds = final_knn_model.predict(test_X)

        final_preds = [mode([i,j,k,l,m])[0][0] for i,j,
                    k,l,m in zip(svm_preds, nb_preds, rf_preds,ds_preds,knn_preds)]

        print(f"Accuracy on Test dataset by the combined model\
        : {accuracy_score(test_Y, final_preds)*100}")


        def predictDisease(symptoms):
            input_data = [0] * len(data_dict["symptom_index"])
            for symptom in symptoms:
                index = data_dict["symptom_index"][symptom]
                input_data[index] = 1
           
            input_data = np.array(input_data).reshape(1,-1)
            # generating individual outputs
            rf_prediction = data_dict["predictions_classes"][final_rf_model.predict(input_data)[0]]
            nb_prediction = data_dict["predictions_classes"][final_nb_model.predict(input_data)[0]]
            svm_prediction = data_dict["predictions_classes"][final_svm_model.predict(input_data)[0]]
            ds_prediction = data_dict["predictions_classes"][final_ds_model.predict(input_data)[0]]
            knn_prediction = data_dict["predictions_classes"][final_knn_model.predict(input_data)[0]]
            
            # making final prediction by taking mode of all predictions
            final_prediction = mode([rf_prediction, nb_prediction, svm_prediction,ds_prediction,knn_prediction])[0][0]
            predictions = {
                "RandomForestClassifier Prediction": rf_prediction,
                "GaussianNB Prediction": nb_prediction,
                "SVC Prediction": svm_prediction,
                "Decision Tree": ds_prediction,
                "KNN Prediction": knn_prediction,
                "Final Prediction":final_prediction
            }
            return predictions

        # Testing the function
        predictiondata = predictDisease(symptomList)
        patient = Patient.objects.get(user=request.user)
        Search_Data.objects.create(patient=patient, prediction_accuracy=round(accuracy_score(test_Y, final_preds)*100,2), result=predictiondata["Final Prediction"], values_list=symptomList, predict_for="General Health Prediction")

        # print(deseaseli)
    alldisease = ['Watering From Eyes','Increased Appetite','Polyuria','Family History','Mucoid Sputum','Rusty Sputum','Irregular Sugar Level','Cough','High Fever','Sunken Eyes','Breathlessness','Sweating','Dehydration','Indigestion','Headache','Yellowish Skin','Dark Urine','Nausea','Loss Of Appetite','Pain Behind The Eyes','Back Pain','Constipation','Abdominal Pain','Diarrhoea','Mild Fever','Yellow Urine','Yellowing Of Eyes','Acute Liver Failure','Fluid Overload','Swelling Of Stomach','Swelled Lymph Nodes','Malaise','Blurred And Distorted Vision','Phlegm','Throat Irritation','Redness Of Eyes','Sinus Pressure','Runny Nose','Congestion','Chest Pain','Weakness In Limbs','Lack Of Concentration','Visual Disturbances','Receiving Blood Transfusion','Receiving Unsterile Injections','Coma','Stomach Bleeding','Distention Of Abdomen','History Of Alcohol Consumption','Fluid Overload','Blood In Sputum','Prominent Veins On Calf','Palpitations','Painful Walking','Pus Filled Pimples', 'Blackheads','Scurring','Skin Peeling','Silver Like Dusting','Small Dents In Nails','Inflammatory Nails','Blister','Itching','Skin Rash','Nodal Skin Eruptions','Continuous Sneezing','Shivering','Chills','Joint Pain','Stomach Pain','Acidity','Ulcers On Tongue','Muscle Wasting','Vomiting','Burning Micturition','Fatigue','Weight Gain','Anxiety','Cold Hands And Feets','Mood Swings','Weight Loss','Restlessness','Lethargy','Patches In Throat','Fast Heart Rate',	'Pain During Bowel Movements','Pain In Anal Region','Bloody Stool','Irritation In Anus','Neck Pain','Dizziness','Cramps','Bruising','Obesity','Swollen Legs','Swollen Blood Vessels','Puffy Face And Eyes','Enlarged Thyroid','Brittle Nails','Swollen Extremeties','Excessive Hunger','Extra Marital Contacts','Drying And Tingling Lips','Slurred Speech','Knee Pain','Hip Joint Pain','Muscle Weakness','Stiff Neck','Swelling Joints','Movement Stiffness','Spinning Movements','Loss Of Balance','Unsteadiness','Weakness Of One Body Side','Loss Of Smell','Bladder Discomfort','Continuous Feel Of Urine','Passage Of Gases','Internal Itching','Toxic Look (Typhos)',	'Depression','Irritability','Muscle Pain','Altered Sensorium','Red Spots Over Body','Belly Pain','Abnormal Menstruation']
    return render(request,'add_genralhealth.html', {'alldisease':alldisease, 'predictiondata':predictiondata})



In [15]:
prdict_heart_disease(['Watering From Eyes','Increased Appetite','Polyuria'])

InvalidIndexError: (slice(None, None, None), -1)